In [1]:
import pandas as pd
import networkx as nx
import json
from IPython.core.display import display, HTML
from string import Template
import pendulum
import copy
%config IPCompleter.use_jedi=False

In [10]:
tx = pd.read_csv('./data/nate_transactions.csv')

In [11]:
tx

,transaction_hash,to_address,from_address,block_timestamp,value,from_is_contract,to_is_contract,nft_details
0,0x009f821f75c9c75a27eddbf0a30f5039dab322e08f31...,0x881d40237659c251811cec9c364ef91dc08d300c,0x2e3922be905a3314fe5e16b44a3e878bae69c150,2021-08-16T23:36:55.000Z,0.000000e+00,NaN,True,NaN
1,0x009f821f75c9c75a27eddbf0a30f5039dab322e08f31...,0x2e3922be905a3314fe5e16b44a3e878bae69c150,0x74de5d4fcbf63e00296fd95d33236b9794016631,2021-08-16T23:36:55.000Z,1.837500e+17,True,NaN,NaN
2,0x00d2b2ba7b09445f49ca8992266acf54fd8e0fd603d4...,0x54ed0feff1467f9d13587dfd39226d08ee1a9606,0x0660459b2b658b232f3db6adff5580e7558f60e6,2021-05-26T22:14:22.000Z,0.000000e+00,NaN,True,NaN
3,0x01627ee26a1ac6ec6a25b6fafcc148e4da59349fe794...,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x0660459b2b658b232f3db6adff5580e7558f60e6,2021-03-13T14:27:11.000Z,1.000000e+18,NaN,True,NaN
4,0x0168a5dafde259a80cb5d77e6c9ff0d446190ddb47d2...,0x8d33303023723de93b213da4eb53be890e747c63,0xf1b30855994a95e7e2024aa44d986dc20fa6f617,2021-07-16T04:57:37.000Z,8.700000e+16,NaN,True,NaN
...,...,...,...,...,...,...,...,...
31706,0xfd46e6a48f3b6d7e0fb34f206999e51008d7b1cad632...,0xe96a1b303a1eb8d04fb973eb2b291b8d591c8f72,0xa6d954d08877f8ce1224f6bfb83484c7d3abf8e9,2018-03-23T09:26:30.000Z,1.000000e+15,True,NaN,NaN
31707,0xfd46e6a48f3b6d7e0fb34f206999e51008d7b1cad632...,0x6be4a7bbb812bfa6a63126ee7b76c8a13529bdb8,0xa6d954d08877f8ce1224f6bfb83484c7d3abf8e9,2018-03-23T09:26:30.000Z,2.000000e+15,True,NaN,NaN
31708,0xfd5264d8b11faa1ca1812a99b250d5dbe66ce37478d8...,0x514910771af9ca656af840dff83e8264ecf986ca,0x49d33037fb37bd8f4a4817bf425540c2d5e4d30e,2021-08-22T23:39:08.000Z,0.000000e+00,NaN,True,NaN
31709,0xfd828fe879d0bf1285986131cf28a9535d7daf9f8980...,0x5fdcca53617f4d2b9134b29090c87d01058e27e9,0xc930830064ca8d752cecb3373197ce8673b21c22,2021-08-23T06:13:19.000Z,3.350000e+17,NaN,True,NaN


In [12]:
G = nx.from_pandas_edgelist(tx, source='from_address', target='to_address', edge_key='transaction_hash', edge_attr=['value', 'block_timestamp', 'nft_details'],
                           create_using=nx.MultiDiGraph(),)

In [14]:
NODE_MAPPING = {
    '0x7be8076f4ea4a4ad08075c2508e481d6c946d12b': 'Opensea Exchange',
    '0xa3a4548b39da96eb065ff91811ca30da40431c0d': 'Nate' #main wallet with CryptoPunk 3501
}

In [15]:
def map_node(n):
    if n in NODE_MAPPING:
        return NODE_MAPPING[n]
    else:
        return n

In [16]:
# find all nodes that are n degrees removed from a given node u in the graph G
def find_paths(G,u,n):
    if n==0:
        return [[u]]
    paths = [[u]+path for neighbor in G.neighbors(u) for path in find_paths(G, neighbor, n-1)]
    return paths

In [17]:
# find all nodes that are n degrees removed from a given node u in the graph G, which form a closed cycle
def find_cycles(G,u,n):
    paths = find_paths(G, u, n)
    return [tuple(path) for path in paths if (path[-1] == u) and sum(x==u for x in path) == 2]